In [1]:
import numpy as np
import torch
from tqdm import tqdm  
import treescope as ts 
import matplotlib.pyplot as plt
from data import DNADataset
dataset = DNADataset("hf://datasets/Zehui127127/latent-dna-diffusion/sequence.csv")

Loading data from hf://datasets/Zehui127127/latent-dna-diffusion/sequence.csv...
Processing data...
Done!


In [2]:
from vqvae import *
model = torch.load(f"model_epoch_{60}.pth", map_location='cpu')
ts.display(model)

/tmp/ipykernel_2011271/4099825031.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f"model_epoch_{60}.pth", map_location='cpu')


In [4]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
for X, y in list(dataloader)[:5]:
    encoded = model.encode(X, c=y)
    decoded = torch.nn.Softmax(-1)(model.decode(encoded, c=y))
    quantized = torch.eye(5)[decoded.argmax(-1)]
    correct = X.argmax(-1) == decoded.argmax(-1)
    ts.display(
        {
            "correct %": 100 * correct.to(float).mean().item(),
            "correct": correct[0],
            "original": X[0].T,
            "decoded": decoded[0].T,
            "quantized": quantized[0].T,
        },
        autovisualize=True,
    )